In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import pandas as pd
from hkfold import HKFold
from sklearn.model_selection import KFold
from keras.layers import PReLU
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dframe_path = 'data/cabauw/processed.csv.gz'

try:
    df = pd.read_csv(dframe_path, na_values='--')
except UnicodeDecodeError:
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')


df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
df = df[df.ds != 201603]
#df = df.sort_values(['ds', 'tt'])
df = df.dropna()
df = df.sample(frac=1)  # aka shuffle

In [3]:
features = [
    'dewpoint', 'spec_hum', 'rel_hum', 'press', 'rain',
    'air_dens', 'wind', 'temp', 'virtual_temp', 'soil_temp', 'z',
    #'ustar', 'H', 'LE', 'zL_m', 'lval'
]

target = 'phi_m'

train_idx, test_idx = next(HKFold(KFold(50, shuffle=True), h=int(1 * 60 / 10)).split(df))

nr, ne, nn = len(train_idx), len(test_idx), len(df)
print('training with %d (%.1f%%) samples, testing with %d (%.1f%%), %d (%.1f%%) samples lost' % (
    nr, 100 * nr / nn, ne, 100 * ne / nn, nn - nr - ne, 100 * (nn - nr - ne) / nn
))

train_x, train_y = df[features].iloc[train_idx], df[target].iloc[train_idx]
test_x, test_y = df[features].iloc[test_idx], df[target].iloc[test_idx]

mean_x, mean_y = train_x.mean(), train_y.mean()
std_x, std_y = train_x.std(), train_y.std()

train_x = (train_x - mean_x) /  std_x
test_x = (test_x - mean_x) / std_x

#train_y = (train_y - mean_y) / std_y
#test_y = (test_y - mean_y) / std_y

training with 1022889 (76.9%) samples, testing with 26602 (2.0%), 280601 (21.1%) samples lost


In [10]:
model = Sequential([
    Dense(1024, input_shape=(len(features),)), PReLU(),
    Dense(1024), PReLU(),
    Dense(256), PReLU(),
    Dense(256), PReLU(),
    Dense(128), PReLU(),
    Dense(128), PReLU(),
    Dense(32), PReLU(),
    Dense(1)
])

opt = RMSprop(lr=0.001)
model.compile(loss='mse', optimizer=opt)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1024)              12288     
_________________________________________________________________
p_re_lu_15 (PReLU)           (None, 1024)              1024      
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
p_re_lu_16 (PReLU)           (None, 1024)              1024      
_________________________________________________________________
dense_19 (Dense)             (None, 256)               262400    
_________________________________________________________________
p_re_lu_17 (PReLU)           (None, 256)               256       
_________________________________________________________________
dense_20 (Dense)             (None, 256)               65792     
__________

In [11]:
hist = model.fit(
    train_x, train_y,
    batch_size=4096, epochs=250,
    callbacks=[
        ReduceLROnPlateau(factor=0.2, verbose=1, min_lr=1e-6),
        ModelCheckpoint('dev/models/weights.{epoch:04d}-{val_loss:.4f}.hdf5',
                        verbose=1, save_best_only=True),
        TensorBoard('dev/logs/', write_graph=False, write_grads=True),
    ],
    validation_data=(test_x, test_y)
)

Train on 1022889 samples, validate on 26602 samples
Epoch 1/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 1.3702
Epoch 00001: val_loss improved from inf to 0.73347, saving model to weights.01-0.73.hdf5
1022889/1022889 [==============================] - 21s 21us/step - loss: 1.3681 - val_loss: 0.7335
Epoch 2/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.6879
Epoch 00002: val_loss improved from 0.73347 to 0.71025, saving model to weights.02-0.71.hdf5
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.6873 - val_loss: 0.7103
Epoch 3/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.5609
Epoch 00003: val_loss improved from 0.71025 to 0.45091, saving model to weights.03-0.45.hdf5
1022889/1022889 [==============================] - 18s 17us/step - loss: 0.5607 - val_loss: 0.4509
Epoch 4/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.5043
Epoch 00004: val_loss improved f

1019904/1022889 [============================>.] - ETA: 0s - loss: 0.1326
Epoch 00034: val_loss did not improve
1022889/1022889 [==============================] - 18s 17us/step - loss: 0.1328 - val_loss: 0.2172
Epoch 35/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.1308
Epoch 00035: val_loss improved from 0.13492 to 0.12200, saving model to weights.35-0.12.hdf5
1022889/1022889 [==============================] - 18s 17us/step - loss: 0.1307 - val_loss: 0.1220
Epoch 36/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.1279
Epoch 00036: val_loss did not improve
1022889/1022889 [==============================] - 18s 17us/step - loss: 0.1279 - val_loss: 0.1376
Epoch 37/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.1248
Epoch 00037: val_loss did not improve
1022889/1022889 [==============================] - 18s 17us/step - loss: 0.1248 - val_loss: 0.1356
Epoch 38/250
1019904/1022889 [============================>.] 

Epoch 69/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0787
Epoch 00069: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0787 - val_loss: 0.0875
Epoch 70/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0777
Epoch 00070: val_loss improved from 0.07906 to 0.07045, saving model to weights.70-0.07.hdf5
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0776 - val_loss: 0.0704
Epoch 71/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0764
Epoch 00071: val_loss did not improve
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0764 - val_loss: 0.0873
Epoch 72/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0764
Epoch 00072: val_loss did not improve
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0764 - val_loss: 0.0874
Epoch 73/250
1019904/1022889 [===================

Epoch 102/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0189
Epoch 00102: val_loss improved from 0.02406 to 0.02374, saving model to weights.102-0.02.hdf5
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0189 - val_loss: 0.0237
Epoch 103/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0185
Epoch 00103: val_loss improved from 0.02374 to 0.02325, saving model to weights.103-0.02.hdf5
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0185 - val_loss: 0.0232
Epoch 104/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0183
Epoch 00104: val_loss did not improve
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0183 - val_loss: 0.0241
Epoch 105/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0180
Epoch 00105: val_loss improved from 0.02325 to 0.02289, saving model to weights.105-0.02.hdf5
1022889/1022889 [==============

Epoch 135/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0130
Epoch 00135: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0130 - val_loss: 0.0174
Epoch 136/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0129
Epoch 00136: val_loss improved from 0.01702 to 0.01657, saving model to weights.136-0.02.hdf5
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0129 - val_loss: 0.0166
Epoch 137/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0129
Epoch 00137: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0129 - val_loss: 0.0176
Epoch 138/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0128
Epoch 00138: val_loss improved from 0.01657 to 0.01533, saving model to weights.138-0.02.hdf5
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0128 - val_l

Epoch 169/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0106
Epoch 00169: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0106 - val_loss: 0.0140
Epoch 170/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0106
Epoch 00170: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0106 - val_loss: 0.0136
Epoch 171/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0105
Epoch 00171: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0105 - val_loss: 0.0139
Epoch 172/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0105
Epoch 00172: val_loss improved from 0.01282 to 0.01274, saving model to weights.172-0.01.hdf5
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0105 - val_loss: 0.0127
Epoch 173/250
1019904/1022889 [=============

Epoch 204/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0092
Epoch 00204: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0092 - val_loss: 0.0114
Epoch 205/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0092
Epoch 00205: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0092 - val_loss: 0.0113
Epoch 206/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0092
Epoch 00206: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0092 - val_loss: 0.0117
Epoch 207/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0091
Epoch 00207: val_loss did not improve
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0091 - val_loss: 0.0118
Epoch 208/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0091- ETA: 0s - l

Epoch 236/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0044
Epoch 00236: val_loss did not improve
1022889/1022889 [==============================] - 17s 17us/step - loss: 0.0044 - val_loss: 0.0062
Epoch 237/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0044
Epoch 00237: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0044 - val_loss: 0.0062
Epoch 238/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0044
Epoch 00238: val_loss improved from 0.00620 to 0.00613, saving model to weights.238-0.01.hdf5
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0044 - val_loss: 0.0061
Epoch 239/250
1019904/1022889 [============================>.] - ETA: 0s - loss: 0.0044
Epoch 00239: val_loss did not improve
1022889/1022889 [==============================] - 17s 16us/step - loss: 0.0044 - val_loss: 0.0061
Epoch 240/250
1019904/1022889 [=============